In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from scipy import stats

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import lxml
import ast
import json

In [ ]:

start_date = dt.datetime.today() - relativedelta(years = 2)
end_date =  dt.datetime.today()

In [ ]:
# dataframe having one row for each date from start date till end date
df_factors = pd.DataFrame({'Business_Date':pd.date_range(start_date, end_date)})

df_factors['Business_Date'] = df_factors['Business_Date'].apply(lambda x : x.date())

In [ ]:
df_factors.head()

Business_Date
0    2018-06-25
1    2018-06-26
2    2018-06-27
3    2018-06-28
4    2018-06-29

## Stock Volume and Adj Close

In [ ]:
df_temp = data.DataReader('AXISBANK.NS', 'yahoo', df_factors['Business_Date'].min(), df_factors['Business_Date'].max())
df_temp['Business_Date'] = pd.to_datetime(df_temp.index)
df_temp['Business_Date'] = df_temp['Business_Date'].apply(lambda x : x.date())

In [ ]:
# Stock Data is being merged with the main data frame
df_factors = pd.merge(df_factors,df_temp[['Business_Date','Volume', 'Adj Close']],on='Business_Date', how='left')

## Nifty index data

In [ ]:
df_nifty = data.DataReader('%5ENSEI', 'yahoo', df_factors['Business_Date'].min(), df_factors['Business_Date'].max())
df_nifty['Business_Date'] = pd.to_datetime(df_nifty.index)
df_nifty['Business_Date'] = df_nifty['Business_Date'].apply(lambda x : x.date())

In [ ]:
# Nifty Data is being merged with the main data frame
df_factors = pd.merge(df_factors,df_nifty[['Business_Date', 'Adj Close']],on='Business_Date', how='left',suffixes=('','_nifty'))

## Nifty Bank index data

In [ ]:
df_banknifty = data.DataReader('%5ENSEBANK', 'yahoo', df_factors['Business_Date'].min(), df_factors['Business_Date'].max())
df_banknifty['Business_Date'] = pd.to_datetime(df_banknifty.index)
df_banknifty['Business_Date'] = df_banknifty['Business_Date'].apply(lambda x : x.date())

In [ ]:
# Nifty Data is being merged with the main data frame
df_factors = pd.merge(df_factors,df_banknifty[['Business_Date', 'Adj Close']],on='Business_Date', how='left',suffixes=('','_banknifty'))

In [ ]:
df_factors.head()

Business_Date     Volume   Adj Close  Adj Close_nifty  Adj Close_banknifty
0    2018-06-25  4882240.0  514.560974     10762.450195         26609.699219
1    2018-06-26  4505122.0  513.562195     10769.150391         26601.699219
2    2018-06-27  5659070.0  507.220093     10671.400391         26423.400391
3    2018-06-28  4854274.0  503.974121     10589.099609         26324.599609
4    2018-06-29  7747444.0  510.166412     10714.299805         26364.199219

In [ ]:
df_factors.tail()

Business_Date      Volume   Adj Close  Adj Close_nifty  Adj Close_banknifty
727    2020-06-21         NaN         NaN              NaN                  NaN
728    2020-06-22  40454004.0  430.149994     10311.200195         21708.349609
729    2020-06-23  30757220.0  443.649994     10471.000000         22264.900391
730    2020-06-24  33577082.0  424.649994     10305.299805         21426.800781
731    2020-06-25  49846496.0  421.700012     10288.900391         21506.150391

## CPI,Money Supply Data,interest rates

In [ ]:
def get_macrodata(indicator, df_input):
    if indicator.lower() == 'inflation':
        url = "https://sbcharts.investing.com/events_charts/us/973.json"
        attr_id =  '973'
    elif indicator.lower() == 'moneysupply':
        url = "https://sbcharts.investing.com/events_charts/us/756.json"
        attr_id =  '756'
    elif indicator.lower() == 'interestrate':
        url = "https://sbcharts.investing.com/events_charts/us/597.json"
        attr_id =  '597'
    else:
        return
    
    col_head = indicator.lower()   
    urlheader = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    payload = {
    'event_attr_ID': attr_id,
    'event_timestamp': dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    
    req = requests.post(url, headers=urlheader, data=payload)
    soup = BeautifulSoup(req.content, "lxml")
    dict = json.loads(soup.p.contents[0])
    df = pd.DataFrame([i[0],i[1]] for i in dict.get('data') )
    df = df.rename(columns = {0:'Business_Date', 1:col_head})
    df['Business_Date'] =  pd.to_datetime(df['Business_Date'], unit='ms')
    df['Business_Date'] = df['Business_Date'].apply(lambda x : x.date())
    

    df_input = pd.merge(df_input,df,on='Business_Date', how='left')
    df_input = df_input.sort_values(by = ['Business_Date'], ascending = True)
    df_input[col_head] = df_input[col_head].fillna(method = 'ffill')
    return df_input
    

In [ ]:
# use get_macrodata(indicator, dataframe) function to get inflation data and merge it with input dataframe, pass indicator argument as 'inflation'
# for moneysupply pass indicator as 'moneysupply'
# for interest rate pass indicator as 'interestrate' 
df_factors = get_macrodata('inflation', df_factors)
df_factors = get_macrodata('moneysupply', df_factors)
df_factors = get_macrodata('interestrate', df_factors)


In [ ]:
df_factors.head()

Business_Date     Volume   Adj Close  ...  inflation  moneysupply  interestrate
0    2018-06-25  4882240.0  514.560974  ...        NaN          NaN           NaN
1    2018-06-26  4505122.0  513.562195  ...        NaN          NaN           NaN
2    2018-06-27  5659070.0  507.220093  ...        NaN          NaN           NaN
3    2018-06-28  4854274.0  503.974121  ...        NaN          NaN           NaN
4    2018-06-29  7747444.0  510.166412  ...        NaN          NaN           NaN

[5 rows x 8 columns]

In [ ]:
df_factors.tail()

Business_Date      Volume   Adj Close  ...  inflation  moneysupply  interestrate
727    2020-06-21         NaN         NaN  ...       5.84         12.4           4.0
728    2020-06-22  40454004.0  430.149994  ...       5.84         12.4           4.0
729    2020-06-23  30757220.0  443.649994  ...       5.84         12.4           4.0
730    2020-06-24  33577082.0  424.649994  ...       5.84         12.4           4.0
731    2020-06-25  49846496.0  421.700012  ...       5.84         12.4           4.0

[5 rows x 8 columns]

In [ ]:

## Gold Rate

In [ ]:
df_gold = data.DataReader('HDFCMFGETF.NS', 'yahoo', df_factors['Business_Date'].min(), df_factors['Business_Date'].max())
df_gold['Business_Date'] = df_gold.index
df_gold['Business_Date'] = df_gold['Business_Date'].apply(lambda x : x.date())

In [ ]:

df_gold.head()

High          Low  ...    Adj Close  Business_Date
Date                                  ...                            
2018-06-25  2814.449951  2795.300049  ...  2803.800049     2018-06-25
2018-06-26  2807.000000  2795.000000  ...  2795.050049     2018-06-26
2018-06-27  2809.250000  2791.000000  ...  2796.350098     2018-06-27
2018-06-28  2816.699951  2788.649902  ...  2795.000000     2018-06-28
2018-06-29  2804.949951  2772.300049  ...  2780.350098     2018-06-29

[5 rows x 7 columns]

In [ ]:
# merging gold data into the main dataframe
df_factors = pd.merge(df_factors,df_gold[['Business_Date', 'Adj Close']],on='Business_Date', how='left', suffixes=('','_gold'))

## Currency Exchange Rate

In [ ]:
df_currency = data.DataReader('INR%3DX', 'yahoo', df_factors['Business_Date'].min() + timedelta(days=1), df_factors['Business_Date'].max() + timedelta(days=1))
df_currency['Business_Date'] = df_currency.index
df_currency['Business_Date'] = df_currency['Business_Date'].apply(lambda x : x.date())

In [ ]:
# merging currency data into the main dataframe
df_factors = pd.merge(df_factors,df_currency[['Business_Date', 'Adj Close']],on='Business_Date', how='left', suffixes=('','_currency'))

In [ ]:
df_m = df_factors
df_m.head()

Business_Date     Volume  ...  Adj Close_gold  Adj Close_currency
0    2018-06-25  4882240.0  ...     2803.800049           68.339996
1    2018-06-26  4505122.0  ...     2795.050049           68.364998
2    2018-06-27  5659070.0  ...     2796.350098           69.239998
3    2018-06-28  4854274.0  ...     2795.000000           68.796700
4    2018-06-29  7747444.0  ...     2780.350098                 NaN

[5 rows x 10 columns]

In [ ]:
df_m['Adj Close_currency'] = df_m['Adj Close_currency'].fillna(method = 'ffill')
df_final = df_m.dropna()
df_final.head()

Business_Date      Volume  ...  Adj Close_gold  Adj Close_currency
37    2018-08-01   6871812.0  ...     2722.449951           68.307098
38    2018-08-02   6495837.0  ...     2713.000000           68.615501
39    2018-08-03  15953749.0  ...     2701.100098           68.615501
42    2018-08-06  22629303.0  ...     2710.000000           68.809998
43    2018-08-07  16424825.0  ...     2705.449951           68.596001

[5 rows x 10 columns]

## Merging tweeter sentiments to the final dataframe

In [ ]:
tweeter_sentiment = pd.read_csv("/content/drive/My Drive/Capstone Project/sentiment score data/Axis_Bank.csv")
tweeter_sentiment.rename(columns= {'Date':'Business_Date','compound':'Sentiment_Score'},inplace=True)
tweeter_sentiment.head()

Business_Date  Sentiment_Score
0    2017-01-01        -0.074000
1    2017-01-02        -0.202300
2    2017-01-03         0.000000
3    2017-01-04         0.382400
4    2017-01-05        -0.094383

In [ ]:
df_final['Business_Date'] = df_final['Business_Date'].apply(str)

In [ ]:
# merging currency data into the main dataframe
df = pd.merge(df_final,tweeter_sentiment[['Business_Date', 'Sentiment_Score']],on='Business_Date', how='left')

In [ ]:
df.head()

Business_Date      Volume  ...  Adj Close_currency  Sentiment_Score
0    2018-08-01   6871812.0  ...           68.307098         0.101237
1    2018-08-02   6495837.0  ...           68.615501         0.190476
2    2018-08-03  15953749.0  ...           68.615501         0.142100
3    2018-08-06  22629303.0  ...           68.809998        -0.202733
4    2018-08-07  16424825.0  ...           68.596001         0.039508

[5 rows x 11 columns]

In [ ]:
filename ='/content/drive/My Drive/Capstone Project/Training Data/'+'AxisBank.csv'
df.to_csv (filename, index = False, header=True)